# Link Prediction in Condmat

In [5]:
from datetime import datetime
import itertools
import math
from typing import List, Any, Dict, Tuple

import joblib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold, cross_validate
import seaborn as sns
from tqdm import tqdm

# Typing
Author = int
Papers = List[Tuple[List[Author], datetime]]
NodePair = Tuple[Author, Author]
Edge = List[Tuple[Author, Author, Dict[str, datetime]]]

folder = '/local/bruingjde/complexnetworks2020-experiment/temp/b1n/'

In [6]:
def _get_papers(filepath: str = "src/cond-mat.hg2") -> Papers:
  """Read collaboration data in filepath and return all papers."""
  
  papers = list()
  # Get number of rows to read for the vertices.
  with open(filepath) as file:
    no_rows = int(file.readline().split(' ')[1])
 
  with open(filepath) as file:
    for paper in file.readlines()[no_rows+2:]:
      # Each line has the following format: epoch no_authors [ u v (w ...) ]
      epoch = datetime.fromtimestamp(int(paper.split(' ')[0]))
          
      no_authors = int(paper.split(' ')[1])
      index1 = paper.find('[')+2
      index2 = paper.find(']')-1

      authors = [int(auth) for auth in paper[index1:index2].split(' ')]
      assert no_authors == len(authors)
      
      papers.append((authors, epoch))
  return papers
def _filter_edgelist(edges: List[Edge], start: float, stop: float) -> List[Edge]:
  no_edges = len(edges)
  start_index = int(start*no_edges)
  start_date = edges[start_index][2]['date']
  stop_index = math.floor(stop*no_edges)-1
  stop_date = edges[stop_index][2]['date']
  return [edge for edge in edges if edge[2]['date'] >= start_date and edge[2]['date'] <= stop_date]
def get_edgelist(*, start=0, stop=1) -> List[Edge]:
  """
  Return edgelist from data. If data is not provided this is automatically 
  created using get_data. **kwargs are provided to this function.
  """
  papers = _get_papers()
  edges = [
    (u, v, dict(date=date)) if u<v else (v, u, dict(date=date))
    for authors, date in papers
    for u, v in itertools.combinations(authors, 2)
  ]
  return _filter_edgelist(edges, start, stop)
def giant_component(graph: nx.Graph) -> nx.Graph: return graph.subgraph(max(nx.connected_components(graph), key=len)).copy()
def get_graph(edgelist: List[Edge]) -> nx.Graph:
  """Add edge to graph. Contains edge attribute weight."""
  g = nx.Graph()
  
  for u, v, _ in edgelist:
    weight = g[u][v]["weight"]+1 if g.has_edge(u,v) else 1
    g.add_edge(u, v, weight=weight)
  
  return g
def report(graph:nx.Graph, probes: Tuple[Author, Author]):
  n = len(probes)
  print(f"Number of probes: {n}")
  a = sum([graph.has_edge(u, v) for u, v in probes])
  print(f"- already edge: {a} ({a/n:.0%})")
  non_edges = set(nx.non_edges(graph))
  ne = sum([np in non_edges for np in probes])
  print(f"- both nodes in graph: {ne} ({ne/n:.0%})")
  ng = sum([not (graph.has_node(u) and graph.has_node(v)) for u, v in probes])
  print(f"- not in graph: {ng} ({ng/n:.0%})")
def get_distances(graph: nx.Graph, cutoff: int = None) -> (List[NodePair], List[int]):
  """
  Get all non-edges using BFS. When cutoff provided, consider only node pairs with at most this distance.
  Returns:
  - nodepairs: tuple containing all nodepairs
  - distances: tuple containing all distances
  """
  return zip(
    *[
      [(u, v), distance]
      for u, (nbs_u, _) in tqdm(nx.all_pairs_dijkstra(graph, cutoff, weight=None), total=len(graph), desc="get_distances")
      for v, distance in nbs_u.items() if distance > 1 and (cutoff is None or distance < cutoff) 
    ]
  )

## Set-up
Choose here the parameters on how you want to define the learn and assessing phase.

In [7]:
g_train_matured = giant_component(get_graph(get_edgelist(stop=1/3)))
uv_train_probe = {(u, v) for u, v, _ in get_edgelist(start=1/3, stop=2/3)}

In [9]:
report(graph=g_train_matured, probes=uv_train_probe)

Number of probes: 21856
- already edge: 3541 (16%)
- both nodes in graph: 2760 (13%)
- not in graph: 15555 (71%)


In [10]:
g_test_matured = giant_component(get_graph(get_edgelist(stop=2/3)))
uv_test_probe = {(u, v) for u, v, _ in get_edgelist(start=2/3)}

In [11]:
report(graph=g_test_matured, probes=uv_test_probe)

Number of probes: 23024
- already edge: 5399 (23%)
- both nodes in graph: 4342 (19%)
- not in graph: 13283 (58%)


## Export

### Train

In [ ]:
nodepairs_train, distances_train = get_distances(g_train_matured)
targets_train = [nodepair in uv_assessing for nodepairs_train in tqdm(nodepairs)]

get_distances:  96%|█████████▌| 6165/6413 [10:17<00:18, 13.34it/s]  

In [ ]:
print(f'{sum(targets_train) / len(nodepairs_train):e}')

In [ ]:
%%time
joblib.dump(targets_train, f'{folder}train/targets.pkl')
joblib.dump(nodepairs_train, f'{folder}train/nodepairs.pkl')
joblib.dump(distances_train, f'{folder}train/distances.pkl')
joblib.dump(g_learn_train, f'{folder}train/graph.pkl')

### Test

In [ ]:
nodepairs_test, distances_test = get_distances(g_test_matured)
targets_test = [nodepair in uv_assessing for nodepairs_test in tqdm(nodepairs)]

In [ ]:
print(f'{sum(targets_test) / len(nodepairs_test):e}')

In [ ]:
%%time
joblib.dump(targets_test, f'{folder}test/targets.pkl')
joblib.dump(nodepairs_test, f'{folder}test/nodepairs.pkl')
joblib.dump(distances_test, f'{folder}test/distances.pkl')
joblib.dump(g_learn_test, f'{folder}test/graph.pkl')